# ch03 Django 웹 프레임워크

## 3.1 일반적인 특징

- MVC 패턴 기반 MTV
- 객체 관계 매핑
- 자동으로 구성되는 관리자 화면
- 우아한 URL 설계
- 자체 템플릿 시스템
- 캐시 시스템
- 다국어 지원
- 풍부한 개발 환경
- 소스 변경사항 자동 반영

## 3.3 장고에서의 애플리케이션 개발 방식

- 웹사이트 설계시 가장 먼저 해야 할 일: 프로그램이 해야 할 일을 적당한 크기로 나누어서 모듈화
- 웹사이트의 전체 프로그램 또는 모듈화 된 단위 프로그램을 애플리케이션이라고 함
- 즉, 프로그램으로 코딩할 대상을 애플리케이션이라고 부름
- 애플리케이션의 개념을 웹 서버 개발 측면에서 좀 더 구체화
- 웹 사이트에 대한 전체 프로그램을 Project
- 모듈화 된 단위 프로그램을 Application
- 즉, 애플리케이션들이 모여서 Project를 개발하는 개념

### 3.3.1 MTV 패턴

- 클라이언트로부터 요청을 받으면 URLconf 모듈을 이용하여 URL을 분석
- URL 분석 결과를 통해 해당 URL에 대한 처리를 담당할 뷰 결정
- 뷰는 자신의 로직을 실행하면서, 만일 데이터베이스 처리가 필요하면 모델을 통해 처리하고 그 결과를 반환 받음
- 뷰는 자신의 로직 처리가 끝나면 템플릿을 사용하여 클라이언트에 전송할 HTML 파일을 생성
- 뷰는 최종 결과로 HTML 파일을 클라이언트에게 보내 응답

### 3.3.2 Model - 데이터베이스 설계
- 모델: 사용될 데이터에 대한 정의를 담고 있는 장고의 클래스

### 3.3.3 Template - 화면 UI 설계

- 템플릿 파일은 \*.html
- 템플릿 파일을 적절한 디렉토리에 위치
- 장고에서 템플릿 파일을 찾는 방식을 이해하고 있어야 하며, 장고는 그에 맞는 위치에 템플릿 파일이 위치해야 템플릿 파일을 찾을 수 있음
- TEMPLATE_DIRS, INSTALLED_APPS 에서 지정된 디렉토리를 검색

### 3.3.4 URLconf - URL 설계

- 파이썬의 URL 지정 방식은 전통적인 자바나 PHP 계열의 URL보다 직관적이고 이해하기 쉬움
- 이런 방식을 Elegant URL
- URL과 처리 함수(View)를 매핑하는 파이썬 코드를 작성해주면 됨
- URL 자체에 처리 함수나 처리용 스크립트 파일 이름이 들어가면 변경이 어려워지기 때문
- 반면 URLconf를 사용하면 URL과 뷰 함수를 서로 자유롭게 연결할 수 있어서 규모가 큰 프로젝트를 개발할 때처럼 URL과 뷰 함수 이름이 자주 바뀌는 경우에도 URLconf에서 매핑한 부분만 수정하면 되므로 변경이 쉬워짐

#### 장고에서 URL을 분석하느 ㄴ순서

1. settings.py 파일의 ROOT_URLCONF 항목을 읽어 URLconf(urls.py)의 위치를 알아냄
2. URLconf 모듈을 로딩하여 urlpatterns 변수에 지정되어 있는 URL 리스트를 검사
3. 위에서부터 순서대로 URL 리스트의 내용을 검사하면서 매치가 되면 검사를 종료
4. 매치가 된 URL의 뷰를 호출. 여기서 뷰는 함수 또는 클래스의 메소드. 호출시 HttpRequest 객체와 매칭할 때 추출된 단어들을 뷰에 인자로 넘겨줌
5. 리스트 끝까지 검사했는데도 매칭에 실패하면 에러를 처리하는 뷰를 호출

### 3.3.5 View - 로직 설계

- 뷰: 웹 요청을 받아서 데이터베이스 접속 등 해당 애플리케이션의 로직에 맞는 처리를 하고, 그 결과 데이터를 HTML로 변환하기 위하여 템플릿 처리를 한 후에, 최종 HTML로 된 응답 데이터를 웹 클라이언트로 반환하는 역할

```python
from django.http import HttpResponse
import datetime

def current_datetime(request):
    now = datetime.datetime.now()
    html = '<html><vody>It is now %s.</body></html>' % now
    return HttpResponse(html)
```

- 뷰 함수는 첫번째 인자: HttpRequest 객체
- 필요한 처리를 한 후에 최종적으로 HttpResponse 객체를 반환
- 만일 에러를 반환하고 싶다면 아래처럼 HttpResponseNotFound와 같은 에러 응답 객체를 반환하면 됨
- 에러 응답 클래스는 모두 HttpResponse 클래스의 하위 클래스로 정의되어 있음

```python
return HttpResponseNotFound('<h1>Page not found</h1>')
```

## 3.4 프로젝트 뼈대 만들기

- 중요한 점: 하나의 애플리케이션이 여러 개의 프로젝트에 포함될 수 있기 때문에, 애플리케이션을 한 번만 개발하고 이르 ㄹ다른 프로젝트에 재사용하여 개발의 생산성을 높일 수 있다는 것
- 또한 애플리케이션 단위로 이들을 모아서 프로젝트로 만들고, 프로젝트를 모아서 더 큰 프로젝트를 만드는 방식으로, 계층적인 웹 프로그램 개발이 가능하다는 장점이 있음

### django 1.7 설치

In [27]:
!pip install -U django==1.7

Requirement already up-to-date: django==1.7 in /Users/re4lfl0w/.virtualenvs/general/lib/python2.7/site-packages
Cleaning up...


In [31]:
!pip freeze | grep Django

Django==1.7


### 3.4.1 프로젝트 생성

In [1]:
!pwd

/Users/re4lfl0w/Documents/ipython/books/django_python_web_programming


In [2]:
!django-admin.py startproject mysite

In [4]:
!ls -l

total 104
-rw-r--r--   1 re4lfl0w  staff  41364  4 16 00:34 ch02.ipynb
-rw-r--r--   1 re4lfl0w  staff   6627  4 16 01:13 ch03.ipynb
drwxr-xr-x   3 re4lfl0w  staff    102  4 15 23:33 ch1
drwxr-xr-x   9 re4lfl0w  staff    306  4 15 23:33 ch2
drwxr-xr-x   7 re4lfl0w  staff    238  4 15 23:33 ch3-4
drwxr-xr-x   7 re4lfl0w  staff    238  4 15 23:33 ch3-6
drwxr-xr-x   7 re4lfl0w  staff    238  4 15 23:33 ch3-7
drwxr-xr-x   7 re4lfl0w  staff    238  4 15 23:33 ch4
drwxr-xr-x   9 re4lfl0w  staff    306  4 15 23:33 ch5-1
drwxr-xr-x   9 re4lfl0w  staff    306  4 15 23:33 ch5-2
drwxr-xr-x  10 re4lfl0w  staff    340  4 15 23:33 ch5-3
drwxr-xr-x  12 re4lfl0w  staff    408  4 15 23:33 ch6-4
drwxr-xr-x  12 re4lfl0w  staff    408  4 15 23:33 ch6-5
drwxr-xr-x   4 re4lfl0w  staff    136  4 16 01:19 mysite


In [6]:
!ls -l mysite

total 8
-rwxr-xr-x  1 re4lfl0w  staff  249  4 16 01:19 manage.py
drwxr-xr-x  6 re4lfl0w  staff  204  4 16 01:19 mysite


In [7]:
!ls -l mysite/mysite/

total 24
-rw-r--r--  1 re4lfl0w  staff     0  4 16 01:19 __init__.py
-rw-r--r--  1 re4lfl0w  staff  1972  4 16 01:19 settings.py
-rw-r--r--  1 re4lfl0w  staff   297  4 16 01:19 urls.py
-rw-r--r--  1 re4lfl0w  staff   387  4 16 01:19 wsgi.py


In [8]:
!mv mysite/ ch3

In [9]:
!ls -l

total 112
-rw-r--r--   1 re4lfl0w  staff  41364  4 16 00:34 ch02.ipynb
-rw-r--r--   1 re4lfl0w  staff   9968  4 16 01:20 ch03.ipynb
drwxr-xr-x   3 re4lfl0w  staff    102  4 15 23:33 ch1
drwxr-xr-x   9 re4lfl0w  staff    306  4 15 23:33 ch2
drwxr-xr-x   4 re4lfl0w  staff    136  4 16 01:19 ch3
drwxr-xr-x   7 re4lfl0w  staff    238  4 15 23:33 ch3-4
drwxr-xr-x   7 re4lfl0w  staff    238  4 15 23:33 ch3-6
drwxr-xr-x   7 re4lfl0w  staff    238  4 15 23:33 ch3-7
drwxr-xr-x   7 re4lfl0w  staff    238  4 15 23:33 ch4
drwxr-xr-x   9 re4lfl0w  staff    306  4 15 23:33 ch5-1
drwxr-xr-x   9 re4lfl0w  staff    306  4 15 23:33 ch5-2
drwxr-xr-x  10 re4lfl0w  staff    340  4 15 23:33 ch5-3
drwxr-xr-x  12 re4lfl0w  staff    408  4 15 23:33 ch6-4
drwxr-xr-x  12 re4lfl0w  staff    408  4 15 23:33 ch6-5


In [13]:
%cd ch3

/Users/re4lfl0w/Documents/ipython/books/django_python_web_programming/ch3


In [14]:
!ls -l

total 8
-rwxr-xr-x  1 re4lfl0w  staff  249  4 16 01:19 manage.py
drwxr-xr-x  6 re4lfl0w  staff  204  4 16 01:19 mysite


In [15]:
!python manage.py startapp polls

In [16]:
!tree

.
├── manage.py
├── mysite
│   ├── __init__.py
│   ├── __init__.pyc
│   ├── settings.py
│   ├── settings.pyc
│   ├── urls.py
│   └── wsgi.py
└── polls
    ├── __init__.py
    ├── admin.py
    ├── models.py
    ├── tests.py
    └── views.py

2 directories, 12 files


In [29]:
!python manage.py migrate

Operations to perform:
  Apply all migrations: admin, contenttypes, auth, sessions
Running migrations:
  Applying contenttypes.0001_initial... OK
  Applying auth.0001_initial... OK
  Applying admin.0001_initial... OK
  Applying sessions.0001_initial... OK


In [45]:
!echo 'db.sqlite3' > .gitignore

In [46]:
!cat .gitignore

db.sqlite3


In [47]:
!git status

On branch master
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	.gitignore

nothing added to commit but untracked files present (use "git add" to track)


In [48]:
!git add .

In [49]:
!git commit -m "Add .gitignore"

[master 2a59fca] Add .gitignore
 1 file changed, 1 insertion(+)
 create mode 100644 .gitignore


In [50]:
!git status

On branch master
nothing to commit, working directory clean


### 3.4.4 지금까지 작업 확인하기

- 0.0.0.0: 현재 명령을 실행중인 서버의 IP 주소가 무엇으로 설정되어 있더라도 그와는 무관하게 웹 접속 요청을 받겠다는 의미
- 즉, 외부에서 runserver를 실행중인 서버의 IP 주소를 브라우저의 주소창에 입력하여 접속하는 것을 가능하게 함

In [58]:
!python manage.py runserver 0.0.0.0:8002

Performing system checks...

System check identified no issues (0 silenced).
April 15, 2015 - 16:44:48
Django version 1.7, using settings 'mysite.settings'
Starting development server at http://0.0.0.0:8002/
Quit the server with CONTROL-C.
[15/Apr/2015 16:44:49] "GET /admin/ HTTP/1.1" 200 2880
[15/Apr/2015 16:44:49] "GET /static/admin/css/base.css HTTP/1.1" 304 0
[15/Apr/2015 16:44:49] "GET /static/admin/css/dashboard.css HTTP/1.1" 304 0
^C


In [57]:
# 상호작용이라 쉘에서 실행해야 됨
#!python manage.py createsuperuser

- Admin 사이트에서 Users와 Groups 테이블을 포함하여 앞으로 만들게 될 테이블에 대한 데이터의 입력, 변경, 삭제 등의 작업을 할 수 있음
- Admin 화면에서 기본적으로 Users와 Groups 테이블이 보이는 것은 이미 settings.py 파일에 django.contrib.admin 애플리케이션이 등록되어 있기 때문

In [61]:
# 프로젝트 뼈대를 만든 후 디렉토리 모습
!tree

.
├── db.sqlite3
├── manage.py
├── mysite
│   ├── __init__.py
│   ├── __init__.pyc
│   ├── settings.py
│   ├── settings.pyc
│   ├── urls.py
│   ├── urls.pyc
│   ├── wsgi.py
│   └── wsgi.pyc
└── polls
    ├── __init__.py
    ├── admin.py
    ├── models.py
    ├── tests.py
    └── views.py

2 directories, 15 files


## 3.5 애플리케이션 개발하기 - 설계

- 애플리케이션의 로직
  - 설문에 해당하는 질문을 보여주고
  - 질문에 포함되어 있는 답변 항목에 투표한 후
  - 투표 결과를 알려주는 예제
  
### 화면 UI 설계

- index.html: 최근에 실시한 질문의 리스트를 보여줍니다.
- detail.html: 하나의 질문에 대해 투표할 수 있도록 답변 항목을 폼으로 보여줍니다.
- results.html: 질문에 따른 투표 결과를 보여줍니다.

### Question 테이블 설계

컬럼명 | 타입 | 제약 조건 | 설명
--- | --- | --- | ---
id | integer | NotNull, PK, AuthIncrement | Primary Key
question_text | varchar(200) | NotNull | 질문 문장
pub_date | datetime | NotNull | 질문 생성 시각

### Choice 테이블 설계

컬럼명 | 타입 | 제약 조건 | 설명
--- | --- | --- | ---
id | integer | NotNull, PK, AutoIncrement | Primary Key
choice_text | varchar(200) | NotNull | 답변 항목 문구
votes | integer | NotNull | 투표 카운트
question_id | integer | NotNull, FK(Question.id), Index | Foreign Key

- Question 테이블: 질문을 저장하는 테이블
- Choice 테이블: 질문별로 선택용 답변 항목을 저장하는 테이블
- 모든 컬럼은 Null을 담을 수 없도록 했음
- Primary Key는 자동 증가 속성으로 지정하였음
- Choice 테이블의 question_id 컬럼은 Question 테이블과 Foreign Key 관계로 연결되도록 했고, 또한 Index를 생성하도록 하였음

## 3.6 애플리케이션 개발하기 - Model 코딩

In [63]:
!git log

commit b30c3498dde87aa58aa6dbb91dce1b5972465664
Author: re4lfl0w <re4lfl0w@gmail.com>
Date:   Thu Apr 16 01:58:21 2015 +0900

    Add polls application and change TIME_ZONE Asia/Seoul

commit 2a59fca629165c6474c346e4b2469357984f63f6
Author: re4lfl0w <re4lfl0w@gmail.com>
Date:   Thu Apr 16 01:36:20 2015 +0900

    Add .gitignore

commit 196a794a8196535ff334cc063be192f480501cdd
Author: re4lfl0w <re4lfl0w@gmail.com>
Date:   Thu Apr 16 01:25:55 2015 +0900

    Add python manage.py startapp polls

commit aca47393f534ff7a8c0638cb8390ccdc32a397a2
Author: re4lfl0w <re4lfl0w@gmail.com>
Date:   Thu Apr 16 01:23:45 2015 +0900

    Initialize. startproject mysite


### 3.6.1 데이터베이스 지정

### 3.6.2 테이블 정의

- 테이블을 하나의 클래스로 정의
- 테이블의 컬럼은 클래스의 변수(속성)로 매핑
- 테이블 클래스는 django.db.models.Model 클래스를 상속받아 정의하고, 각 클래스 변수의 타입도 장고에서 미리 정의된 필드 클래스를 사용

#### Question 테이블 컬럼과 클래스 변수간의 매핑

테이블 컬럼명 | 컬럼 타입 | 장고의 클래스 변수 | 장고의 필드 클래스
--- | --- | --- | ---
id | integer | (question_id) | (PK는 장고에서 자동 생성해줌)
question_text | varchar(200) | question_text | models.CharField(max_length=200)
pub_date | datetime | pub_date | models.DateTimeField('date published')

#### Choice 테이블 컬럼과 클래스 변수간의 매핑

컬럼명 | 타입 | 장고의 클래스 변수 | 장고의 필드 클래스
--- | --- | --- | ---
id | integer | (choice_id) | (PK는 장고에서 자동 생성해줌)
choice_text | varchar(200) | choice_text | models.ForeignKey(Question)
votes | integer | votes | models.CharField(max_length=200)
question_id | integer | question_id | models.IntegerField(default=0)

#### 유의할 사항

- PK는 클래스에 지정해주지 않아도, 장고는 항상 PK에 대한 속성을 Not Null 및 AutoIncrement로, 이름은 테이블명의 소문자를 접두어로 해서 자동으로 만들어 줌
- DateTimeField() 필드 클래스에 정의한 date published는 pub_date 컬럼에 대한 레이블 문구임. 나중에 설명하는 Admin 사이트에서 이 문구를 보게 될 것임
- FK(Foreign Key)는 항상 다른 테이블의 PK에 연결되므로, Question 클래스의 question_id까지 지정할 필요없이 Question 클래스만 지정하면 됨
- \_\_unicode\_\_() 함수는 객체를 스트링으로 표현할 때 사용하는 함수. 나중에 보게 될 Admin 사이트나 장고 쉘 등에서 테이블명을 보여줘야 하는데, 이때 \_\_unicode\_\_() 함수를 정의하지 않으면 테이블명이 제대로 표시되지 않음. 참고로 파이썬 3에서는 \_\_str\_\_() 함수로 변경 됨

### 3.6.3 Admin 사이트에 테이블 반영

- models.py 파일에서 정의한 테이블도 Admin 사이트에 보이도록 등록
- models.py 모듈에서 정의한 Question, Choice 클래스를 import 하고, admin.site.register() 함수를 사용하여 임포트한 클래스를 Admin 사이트에 등록해주면 됨
- 테이블을 새로 만들 때는 models.py 와 admin.py 2개의 파일을 함께 수정해야 한다는 것 기억

In [64]:
!git status

On branch master
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   polls/admin.py
	modified:   polls/models.py

no changes added to commit (use "git add" and/or "git commit -a")


In [65]:
!git add .

In [66]:
!git commit -m "Add Question and Choice model"

[master 47cc44b] Add Question and Choice model
 2 files changed, 19 insertions(+), 2 deletions(-)


### 3.6.4 데이터베이스 변경사항 반영

- 테이블의 신규 생성, 테이블의 정의 변경 등 데이터베이스에 변경이 필요한 사항이 있으면, 이를 데이터베이스에 실제로 방영해주는 작업
- 아직까지는 클래스로 테이블 정의만 변경한 상태
- 다음 명령으로 변경사항을 데이터베이스에 반영

In [71]:
!python manage.py makemigrations

No changes detected


In [72]:
!python manage.py migrate

Operations to perform:
  Synchronize unmigrated apps: polls
  Apply all migrations: admin, contenttypes, auth, sessions
Synchronizing apps without migrations:
  Creating tables...
  Installing custom SQL...
  Installing indexes...
Running migrations:
  No migrations to apply.


- makemigrations: polls/migrations 디렉토리 하위에 마이그레이션 파일들이 생기고
- migrate: 이 마이그레이션 파일들을 이용하여 데이터베이스에 테이블을 만들어 줌

#### 장고가 사용하는 SQL 문장 확인

```bash
$ python manage.py sqlmigrate polls 0001
```

In [76]:
!python manage.py sqlmigrate polls 0001

CommandError: App 'polls' does not have migrations


### 3.7 애플리케이션 개발하기 - View 및 Template 코딩

- 사용자에게 보여지는 페이지가 3개이므로, 3개의 템플릿 파일이 필요함
- 중요한 점: URL과 뷰는 항상 1:1 관계로 매핑됨

### 표 3-7 URLconf 설계 - URL과 뷰 매핑

URL 패턴 | 뷰 이름 | 뷰가 처리하느 ㄴ내용
--- | --- | ---
/polls/ | index() | index.html 템플릿을 보여줌
/polls/5/ | detail() | detail.html 템플릿을 보여줌
/polls/5/vote/ | vote() | detail.html에 있는 폼을 POST 방식으로 처리
/polls/5/results/ | results() | results.html 템플릿을 보여줌
/admin/ | (장고 기능) | Admin 사이트를 보여줌(장고에서 기본으로 제공함)

- 로직의 흐름상 URLconf를 먼저 코딩한 후에 뷰, 템플릿 순서로 코딩

#### 로직

1. urls.py 작성 // URLconf 내용을 코딩
2. views.index() 함수 작성 // index.html 템플릿도 같이 작성
3. views.detail() 함수 작성 // detail.html 템플릿도 같이 작성
4. views.vote() 함수 작성 // 리다이렉션 처리 들어있음
5. views.results() 함수 작성 // results.html 템플릿도 같이 작성

### 3.7.1 URLconf 코딩

```python
url(regex, view, kwargs=None, name=None, prefix='')
```

- regex: URL을 정규표현식으로 표현. 정규표현식을 통해 뷰 함수에 넘겨줄 파라미터를 추출할 수 있음
- view: 요청의 URL이 regex 인자에 매칭되면 장고가 뷰 함수를 호출함. 뷰 함수에는 HttpRequest와 regex에서 추출한 항목을 인자로 넘겨줌
- kwargs: regex 인자에서 추출한 파라미터 외에 추가적인 인자를 파이썬 사전 타입의 키워드 인자로 뷰 함수에 넘겨줄 수 있음
- name: 각 URL별로 이름을 붙여줌. 여기서 정해준 이름은 템플릿 파일에서 사용되니 기억해 두기 바람
- prefix: 뷰 함수에 대한 접두사 문자열임. 우리 예제에서는 사용하지 않으니 무시해도 됨

- 2번째 방법 추천
- 즉, URLconf 모듈을 계층적으로 구성
- 더 중요한 사항은 나중에 yoursite라는 프로젝트에 polls 애플리케이션을 재사용하는 경우, 하위의 URLconf를 그대로 가져가서 사용할 수 있다는 것. 이것이 재사용을 기본 원칙으로 하는 장고의 장점 중에 하나
- namespace 인자는 URL 패턴의 이름이 충돌나는 것을 방지하기 위한 것
- polls 애플리케이션의 URL 패턴 이름과 blog 애플리케이션의 URL 패턴 이름이 모두 detail 이 되는 경우가 발생할 수 있음
- 이 둘을 구별하기 위해 include() 함수에 namespace 인자를 사용하는 것
- 즉, polls 애플리케이션의 detail은 **polls\:detail**, blog 애플리케이션의 detail은 **blog\:detail** 로 표기해서 구분

### 3.7.2 뷰 함수 및 index() 및 템플릿 작성

- 화면에 질문을 보여주는 로직을 구현하기 위해 뷰 함수와 템플릿 파일을 코딩하는 것
- 뷰 함수와 템플릿음 서로에게 영향을 미치기 때문에 보통 같이 작업하게 됨
- 또한, UI 화면을 생각하면서 코딩하게 되므로 일반적으로 템플릿을 먼저 코딩하는 것이 정석


- 여기서 중요한 점: 뷰 함수에서 어떤 파라미터를 템플릿으로 넘겨줄지 결정하는 것
- 우리 예제에서는 질문으로 사용될 여러개의 **question_text**를 화면에 보여줘야 하고
- 또한 URL 링크 연결을 위해 템플릿에서 **question.id**도 필요함
- 이 2가지 정보가 함께 들어있는 Question 객체를 넘겨주면 됨
- 이 사항을 index() 뷰 함수에 코딩하겠음

- 뷰에서 지정하는 템플릿의 위치: TEMPLATE_DIRS, INSTALLED_APP에서 지정된 디렉토리를 검색
- 애플리케이션 개수가 늘어나고 템플릿 파일도 점점 많아지면, 애플리케이션은 다르지만 템플릿 파일 이름이 같은 경우가 발생할 수 있음
- 이런 경우 템플릿 파일이 충돌나는 것을 방지하기 위하여 templates 디렉토리 하위에 다시 애플리케이션명으로 디렉토리를 만들어 템플릿 파일을 위치시키는 방법을 사용함
- ~/ch3/polls/templates/polls/index.html
- **단축 함수(shortcut)**: 웹 프로그램 개발 시 자주 사용되는 기능들, 예를 들어 템플릿 코드를 로딩한 후에 컨텍스트 변수를 적용하고, 그 결과를 HTTPResponse 객체에 담아 반환하는 작업 등의 공통적으로 사용되는 기능들을 장고에서는 이미 개발하여 내장 함수로 제공하고 있음

In [78]:
!tree

.
├── db.sqlite3
├── manage.py
├── mysite
│   ├── __init__.py
│   ├── __init__.pyc
│   ├── settings.py
│   ├── settings.pyc
│   ├── urls.py
│   ├── urls.pyc
│   ├── wsgi.py
│   └── wsgi.pyc
└── polls
    ├── __init__.py
    ├── __init__.pyc
    ├── admin.py
    ├── admin.pyc
    ├── models.py
    ├── models.pyc
    ├── templates
    │   └── polls
    │       └── index.html
    ├── tests.py
    ├── urls.py
    ├── urls.pyc
    ├── views.py
    └── views.pyc

4 directories, 22 files


### 3.7.3 뷰 함수 detail() 및 폼 템플릿 작성

- \<form action\> 속성에 {% url %} 템플릿 태그를 사용하여 받을 곳의 URL을 **polls\:vote**로 지정. URL에 대한 이름 공간(NameSpace)은 include() 함수에 정의되어 있음.
- question.choice_set.all: Question 객체의 hoice_set 속성에 들어있는 항목 모두를 뜻함
- label for, input id: label for가 설정되어 있으면 text를 클릭해도 radio 버튼이 선택이 되니 마우스 유저들이 편하다.
- questiond_id 파라미터가 뷰 함수의 인자로 넘어오는 것

```python
url(r'^polls/(?P\<question_id\>\d+)/$', views.detail, name='detail'),
```

- get_object_or_404: 단축함수를 사용하고 있음. 이 함수의 첫 번째 인자는 모델 클래스이고, 두 번째 인자부터는 검색 조건을 여러 개 사용할 수 있음. 위 예제에서는 Question 모델 클래스로부터 pk=question_id 검색 조건에 맞는 객체를 조회함. 조건에 맞는 객체가 없으면 Http404 Exception을 발생시킴

> 단축함수: 대상 객체를 리스트로 가져오는 get_list_or_404() 단축함수도 있음. get_object_or_404() 단축 함수는 get() 함수를 사용하는 데 비해, get_list_or_404() 단축함수는 filter() 함수를 사용하고 또한 리스트가 비어 있으면 Http404 Exception을 발생시킴

### 3.7.4 뷰 함수 vote() 및 리다이렉션 작성

- vote() 뷰 함수의 호출과 연계된 URL은 detail.html 템플릿 파일에서 받음
- 즉 detail.html 템플릿에 있는 폼을 제출하면 폼의 기능에 의해 /polls/5/vote 와 같은 URL이 POST 방식으로 넘어옴
- HttpResponseRedirect: 리다이렉트
- reverse(): URL 패턴으로부터 URL tmxmflddmf rngkf tn dlTdma

```python
reverse('polls:results', args=(p.id,))
```

- reverse() 함수를 사용하여 URL을 구하는 것은 URLconf 에 이미 정의된 URL 패턴에서 URL 스트링을 추출하는 방식이므로, 소스에 URL 스트링을 하드코딩하지 않도록 함


    /polls/<question_id>/results/    ->   views.results()
    reverse() 함수로 URL 추출 
    /polls/3/results/   <-   reverse('polls:results', args=(3,))

### 3.7.5 뷰 함수 results() 및 템플릿 작성

- vote{{ choice.votes|pluralize }}: choice.votes 값에 따라 복수 접미사(s)를 붙여주는 것. 결과적으로 choice.votes 값에 따라 vote 또는 votes가 표시됨
- URL 스트링 추출
```python
{% url 'polls:detail' question.id %}         # 템플릿에서 사용됨
reverse('polls:detail', args=(question_id,)) # 뷰 함수에서 사용됨
```

### 3.7.6 지금까지 작업 확인하기

- 각종 오타와 복붙했을 때 수정하지 않은 것들 때문에 오류가 많이 발생